# I. Introductions

# II. Functions vs Generators (vs Generator Expressions vs Iterables)

## Functions

A typical function definition looks like the below. In this case, we return two values as a tuple. We use this function by calling it with ().

In Python, functions are generalised as "callables" and support the __call__ protocol. Many different things are callables (classes, C-functions, generators, &c.)

In [ ]:
def function(x):
    return x+1, x+2

assert function(10) == (11,12)

# example usage
answer = function(10)
print('function(10) -> {}'.format(answer))

## Generators

A typical generator definition looks like the below. We swap out the `return` statement with a `yield statement` and provide two values. We use this generator by instantiating it with () then iterating over it.

In Python, generators are generalised as "iterables" and support the __iter__/__next__ protocols. Many different things are iterable (lists, tuples, dictionaries, generators, &c.)

In [ ]:
def generator(x):
    yield x+1
    yield x+2

assert list(generator(10)) == [11, 12]

# example usage
for x in generator(10):
    print('generator(10) -> {}'.format(x))

## Generator Expressions

We may note that a generator defined with `yield`-syntax appears to support both the __call__ and __iter__/__next__ protocols. In this case, we __call__ the generator to create an instance, then we iterate over the instance.

A generator expression is one way we can construct a single instance inline.

In [ ]:
generator_expression = (x+1 for x in [1,2,4,8,16])

assert list(generator_expression) == [2,3,5,9,17]

In [ ]:
generator_expression = (x+1 for x in [1,2,4,8,16])

# example usage
for x in generator_expression:
    print('generator_expression -> {}'.format(x))

A generator defined with `yield`-syntax is not strictly the same as, but can be considered equivalent with a generator expression that is constructed dynamically behind a lambda.

In [ ]:
generator_expression = lambda: (x+1 for x in [1,2,4,8,16])

assert list(generator_expression()) == [2,3,5,9,17]

# example usage
for x in generator_expression():
    print('generator_expression() -> {}'.format(x))

## Iterables

Callables are the Python generalisation of functions. 

In Python a callable is just some object that supports the __call__ protocol. __call__ corresponds to () or apply()

In [ ]:
class Callable(object):
    def __call__(self, x):
        return x + 1, x + 2

assert Callable()(10) == (11, 12)

# exampe usage
answer = Callable()(10)
print('Callable(10) -> {}'.format(answer))

What if `__call__` is a staticmethod?

In [ ]:
class Callable(object):
    @staticmethod
    def __call__(x):
        return x + 1, x + 2

assert Callable()(10) == (11, 12)

In [ ]:
from collections import Callable

def function():
    pass

assert callable(function) # deprecated in Python 3
assert isinstance(function, Callable)

Iterables are the Python generalisation of lists, tuples, &c.

In Python an iterable is just some object that supports the `__iter__` protocol and returns an object which supports the `__next__` protocol. `__iter__` corresponds to `iter()` and `__next__` corresponds to `next()`

Note that in Python 2, the __next__ protocol is provided by next. This was fixed in Python 3 (which we should all be using.)

In [ ]:
some_list = [1, 1, 2, 3, 5]

for x in some_list:
    print('some_list -> {}'.format(x))

In [ ]:
class Iterable(object):
    def __init__(self, x):
        self.x = x
        self.state = 0
    def __iter__(self):
        return self # this object is both an iterator and an iterable
    def __next__(self):
        if self.state == 2:
            raise StopIteration
        rv = self.x + self.state
        self.state += 1
        return rv
    next = __next__ # Python 3 calls it `__next__`; Python 2 calls it `next`
    
assert list(Iterable(10)) == [10, 11]

# example usage
for x in Iterable(10):
    print 'Iterable(10) -> {}'.format(x)

Note that, in the above, we have an instance member called `self.state` that tracks the state.

An __iterator__ is anything that can be iterated over.

An __iterable__ is merely some object that tracks the state of the iteration.

In [ ]:
from collections import Iterator, Iterable

some_list = [1, 1, 2, 3, 5]
assert isinstance(some_list, Iterable) and not isinstance(some_list, Iterator)

some_iter = iter(some_list)
assert isinstance(some_iter, Iterable) and isinstance(some_iter, Iterator)

A generator is merely a much lower-level expression of the above.

Instead of explicitly tracking the state with some instance member, the state is tracked by a frame object which tracks the last line of code executed.

# III. Coroutines

The generator below can `yield` values. This is in correspondence to how a function can `return` values.

In [1]:
def generator(x):
    yield x+1
    yield x+2

assert list(generator(10)) == [11, 12]

We can also _send_ values into a generator at any point during the iteration. This is what makes a generator a __coroutine__.

In fact, generators support a rich protocol including:

- `.__next__`: get the next value
- `.throw`: raise an exception
- `.send`: send a value in
- `.close`: terminate iteration

## `next(g)` & `g.__next__()`

In [3]:
def generator(x):
    yield x+1
    yield x+2

assert list(generator(10)) == [11, 12]
    
# support for next(), .next()
g = generator(10)
print('next(g) -> {}'.format(next(g)))  # these two lines are 
print('next(g) -> {}'.format(g.__next__()))

# next too many times, and we get a StopIteration
try:
    next(g)
except StopIteration as e:
    print('next(g) -> {!r}'.format(e))

next(g) -> 11
next(g) -> 12
next(g) -> StopIteration()


## .throw

In [7]:
from sys import exc_info
from traceback import print_tb

def generator(x):
    yield x+1
    yield x+2

g = generator(10)
print('next(g) -> {}'.format(next(g)))
# print 'next(g) -> {}'.format(next(g))

# support for .throw
try:
    g.throw(ValueError('raised value error'))
except ValueError as e:
    print('g.throw(ValueError) -> {!r}'.format(e))
    _, _, traceback = exc_info()
    print_tb(traceback)

next(g) -> 11
g.throw(ValueError) -> ValueError('raised value error',)


  File "<ipython-input-7-2d7f029bd0ff>", line 14, in <module>
    g.throw(ValueError('raised value error'))
  File "<ipython-input-7-2d7f029bd0ff>", line 5, in generator
    yield x+1


## .close

In [9]:
def generator(x):
    yield x+1
    yield x+2

g = generator(10)
print('next(g) -> {}'.format(next(g)))

g.close()

try:
    next(g)
except StopIteration as e:
    print('g.close()')
    print('next(g) -> {!r}'.format(e))

next(g) -> 11
g.close()
next(g) -> StopIteration()


## .send

`next(g)` is the same as `g.__next__()`

These are the same as `g.send(None)` or `g.send()`

The `g.send()` method returns the next value yielded by the generator, or raises `StopIteration` if the generator exits without yielding another value. 

In [41]:
def generator(x):
    y = None
    y = yield x+1, y
    y = yield x+2, y

g = generator(10)
print('next(g) -> {}'.format(g.__next__()))
print('g.send("abc") -> {}'.format(g.send("abc")))
try:
    g.send("def")
except StopIteration as e:
    print('send(g) -> {!r}'.format(e))

next(g) -> (11, None)
g.send("abc") -> (12, 'abc')
send(g) -> StopIteration()


## pumping & priming

One annoyance of generators is that we retrieve a value sent into the generator on the left hand side of a `yield`.

This means that we must `yield` a value before we can accept a value back in.

As a result, we have the following problem.

In [49]:
def generator(x):
    yield x + 1
    
g = generator(10)
g.send(10)

TypeError: can't send non-None value to a just-started generator

# IV. Itertools

Itertools is an extremely useful collection of utilities in the standard library.

It contains very efficient generalisations, helpers, and algorithms that allow us to work very effectively with generators.

The contents of itertools are themselves not generators; they are written in C and are mostly standard Python C-functions and C-objects.

## generalisation

In [52]:
# an example of a generalisation: slicing
from itertools import islice

# standard lists can be sliced
some_list = [1, 1, 2, 3, 5, 8]
print(some_list[1:5])

# abstract iterables can be isliced
def fibonacci(a=1, b=1):
    while True:
        yield a
        a, b = b, a+b

print(list(islice(fibonacci(),1,5)))

# other examples: izip, imap, ifilter (Python2)

[1, 2, 3, 5]
[1, 2, 3, 5]


## helper

In [55]:
# an example of a helper: chain
from itertools import chain, islice

# standard lists can be appended
some_list = [1, 1, 2, 3, 5, 8]
some_list = [0] + some_list
print(some_list)

# abstract iterables can be chained
def fibonacci(a=1, b=1):
    while True:
        yield a
        a, b = b, a+b
    
f = fibonacci()
f = chain([0], f)

print(list(islice(f,0,7)))

# other examples: tee, repeat, cycle

[0, 1, 1, 2, 3, 5, 8]
[0, 1, 1, 2, 3, 5, 8]


## algorithm

In [57]:
# an example of an algorithm: takewhile
from itertools import takewhile

# take elements as long as some predicate is True
def fibonacci(a=1, b=1):
    while True:
        yield a
        a, b = b, a+b

print(list(takewhile(lambda n: n < 50, fibonacci())))

# other examples: dropwhile, product, combinations, permutations

[1, 1, 2, 3, 5, 8, 13, 21, 34]


# V. Efficiency

One reason to use generators is that they can efficiently represent certain constructs.

## memory efficiency

For algorithms that require the use of only a subset of the entire return value, generators can prove far more memory efficient than their functional equivalents.

In [59]:
def pairwise(xs):
    xs = iter(xs)
    pairings = []
    x = next(xs)
    for y in xs:
        pairings.append([x,y])
        x = y
    return pairings

print(pairwise(range(10)))
# print pairwise(xrange(1000000))

[[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9]]


In [62]:
def pairwise(xs):
    xs = iter(xs)
    x = next(xs)
    for y in xs:
        yield x, y
        x = y

print(list(pairwise(range(10))))
# print pairwise(xrange(1000000))

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]


In [66]:
from itertools import tee, islice
nwise = lambda g,n=2: zip(*(islice(g,i,None) for i,g in enumerate(tee(g,n))))

print(list(nwise(range(10))))

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]


In [98]:
def nwise2(g, n=2):
    for i, g in enumerate(tee(g, n)):
        yield islice(g, i, None)
list(zip(*nwise2(range(10))))

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]

## time efficiency

Generators tend to be fairly efficient in practice: <performance/bythrees> vs <performance/bythrees.py>

In [100]:
from math import ceil, sqrt
isprime = lambda n: 1 < n < 4 or all(n % d for d in range(2,int(ceil(sqrt(n)))+1,2))
   
%timeit -n100 isprime(11984395091324)

3.5 µs ± 1.75 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [102]:
from math import ceil, sqrt
def isprime(n):
    if 1 < n < 4:
        return True
    for d in range(2, int(ceil(sqrt(n)))+1, 2):
        if n % d == 0:
            return False
    return True

%timeit -n100 isprime(11984395091324)

1.3 µs ± 76.2 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [103]:
from math import ceil, sqrt
isprime = lambda n, ng: 1 < n < 4 or all(ng)
   
n = 11984395091324
ng = (n % d for d in range(2,int(ceil(sqrt(n)))+1,2))

%timeit -n100 isprime(n, ng)

The slowest run took 8923.94 times longer than the fastest. This could mean that an intermediate result is being cached.
343 µs ± 840 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## structural considerations

Generators are opaque building-blocks, so they can be swapped out very easily with more efficient representations: <nwise_main.py>

In [113]:
#!/usr/bin/env python

if __name__ == '__main__':
    from sys import stdin
    print("\n".join(",".join(x) for x in nwise(y.strip() for y in stdin)))

# VI. Modelling

Someone asked me about _numpy arrays_ versus _generators_.

Numpy arrays are both efficient in practice and also very convenient to use.

While generators can be very efficient and convenient, their real draw is in the ability to better model problems.

## stream data processing & pipeline flow

Generators allow us to model programmes with a stream processing or pipeline conceptualisation.

## presumptions about return type

In [119]:
def fibonacci(n, a=1, b=1):
    rv = [a, b]
    while rv[-1] + rv[-2] < n:
        rv.append(rv[-1] + rv[-2])
    return rv

print(fibonacci(20))
print(set(fibonacci(20)))
print(tuple(fibonacci(20)))

%timeit fibonacci(20000)
%timeit set(fibonacci(20000))
%timeit tuple(fibonacci(20000))

[1, 1, 2, 3, 5, 8, 13]
{1, 2, 3, 5, 8, 13}
(1, 1, 2, 3, 5, 8, 13)
5.73 µs ± 28.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
7 µs ± 65.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
6.03 µs ± 50 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [118]:
def fibonacci(n, a=1, b=1):
    while a < n:
        yield a
        a, b = b, a+b

print(list(fibonacci(20)))
print(set(fibonacci(20)))
print(tuple(fibonacci(20)))

%timeit set(fibonacci(20000))
%timeit list(fibonacci(20000))
%timeit tuple(fibonacci(20000))

[1, 1, 2, 3, 5, 8, 13]
{1, 2, 3, 5, 8, 13}
(1, 1, 2, 3, 5, 8, 13)
3.75 µs ± 20.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.14 µs ± 12.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.11 µs ± 49.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## presumptions about return values

In [124]:
# fibonacci numbers up to but not exceeding n
def fibonacci(n, a=1, b=1):
    rv = [a, b]
    while rv[-1] + rv[-2] < n:
        rv.append(rv[-1] + rv[-2])
    return rv
        
print(fibonacci(20))

[1, 1, 2, 3, 5, 8, 13]


In [137]:
# first n values
def fibonacci(n, a=1, b=1):
    rv = [a, b]
    for _ in range(2,n):
        rv.append(rv[-1] + rv[-2])
    return rv

print(fibonacci(20))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]


In [138]:
from itertools import takewhile, islice

def fibonacci(a=1, b=1):
    while True:
        yield a
        a, b = b, a+b

print(list(islice(fibonacci(), 0, 20))) # first twenty values
print(list(takewhile(lambda n: n < 20, fibonacci()))) # values up to but not exceeding 20

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]
[1, 1, 2, 3, 5, 8, 13]


## presumptions about use of values

In [142]:
from time import sleep

# fibonacci numbers up to but not exceeding n
def fibonacci(n, a=1, b=1):
    rv = [a, b]
    for _ in range(2,n):
        rv.append(rv[-1] + rv[-2])
        sleep(0.01)
    return rv

print(fibonacci(20)[0])
%timeit fibonacci(20)[0]

1
182 ms ± 42.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [150]:
from time import sleep

def fibonacci(a=1, b=1):
    while True:
        yield a
        a, b = b, a+b
        sleep(.01)

print(next(fibonacci()))
%timeit next(fibonacci())

1
435 ns ± 3.62 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [152]:
from datetime import date, timedelta
from time import sleep

holidays = { 'new years':        date(2013, 1, 1),
             'mlk day':          date(2013, 1, 21),
             'presidents day':   date(2013, 2, 18),
             'good friday':      date(2013, 3, 29),
             'memorial':         date(2013, 5, 27),
             'independence day': date(2013, 6, 4),
             'labour day':       date(2013, 9, 2),
             'columbus day':     date(2013, 10, 14),
             'veterans day':     date(2013, 11, 11),
             'thanksgiving':     date(2013, 11, 29),
             'christmas':        date(2013, 12, 24) }

def next_business_day(refdate, n=1, holidays=set(holidays.values())):
    sleep(.01)
    while refdate.weekday() in (5,6) or refdate in holidays:
            refdate += timedelta(days=1)    
    while n:
        refdate += timedelta(days=1)
        while refdate.weekday() in (5,6) or refdate in holidays:
            refdate += timedelta(days=1)
        n -= 1
    return refdate

#         July                 August              September        
# Su Mo Tu We Th Fr Sa  Su Mo Tu We Th Fr Sa  Su Mo Tu We Th Fr Sa  
#     1  2  3  4  5  6               1  2  3   1  2  3  4  5  6  7  
#  7  8  9 10 11 12 13   4  5  6  7  8  9 10   8  9 10 11 12 13 14  
# 14 15 16 17 18 19 20  11 12 13 14 15 16 17  15 16 17 18 19 20 21  
# 21 22 23 24 25 26 27  18 19 20 21 22 23 24  22 23 24 25 26 27 28  
# 28 29 30 31           25 26 27 28 29 30 31  29 30    

print(next_business_day(date(2013, 8, 30)))
print(next_business_day(date(2013, 9, 1)))

def next_business_days(refdate, n=10):
    next_days = [next_business_day(refdate)]
    for _ in range(n-1):
        next_days.append(next_business_day(ten_days[-1]))
    return next_days

print(next_business_days(date(2013, 1, 1), 10))
%timeit next_business_days(date(2013, 1, 1), 90)

2013-09-04


NameError: name 'ten_days' is not defined

In [154]:
def next_business_days(refdate, holidays=set(holidays.values())):  
    sleep(.01)
    while refdate.weekday() in (5,6) or refdate in holidays:
            refdate += timedelta(days=1)        
    while True:
        refdate += timedelta(days=1)
        while refdate.weekday() in (5,6) or refdate in holidays:
            refdate += timedelta(days=1)
        yield refdate

print(list(islice(next_business_days(date(2013, 1, 1)), None, 10)))
%timeit list(islice(next_business_days(date(2013, 1, 1)), None, 90))

[datetime.date(2013, 1, 3), datetime.date(2013, 1, 4), datetime.date(2013, 1, 7), datetime.date(2013, 1, 8), datetime.date(2013, 1, 9), datetime.date(2013, 1, 10), datetime.date(2013, 1, 11), datetime.date(2013, 1, 14), datetime.date(2013, 1, 15), datetime.date(2013, 1, 16)]
10.5 ms ± 63.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## knobs and buttons and modalities

In [155]:
from functools import wraps

# messy
def pumped(gen):
    @wraps(gen)
    def inner(*args, **kwargs):
        g = gen(*args, **kwargs)
        next(g)
        return g.send
    return inner

In [156]:
@pumped
def predicate(x, state=0):
    value = yield
    while True:
        if state+value <= x: # take values until you exceed the maximum
            state += value
            value = yield True 
        else:
            value = yield False

from itertools import repeat, chain, takewhile
greedy = lambda items, predicate: chain.from_iterable(takewhile(predicate,repeat(x)) for x in items)

In [159]:
from __future__ import division, unicode_literals
from itertools import groupby
from random import randint

denominations = [1,5,10,25,100,500,1000,2000]

for _ in range(10):
    amount = randint(0,1000) # randomly pick a dollar amount
    pred = predicate(amount) # create the predicate

    coins = greedy(reversed(denominations), pred) # greedy algorithm

    # pretty print
    print('your change for {:>5.2f}$ = {}'.format(amount/100, 
          ' + '.join('{:>2d}×{:<3}'.format(sum(1 for _ in cs),
          (('{:d}¢' if c < 100 else '{:.0f}$').format(c if c < 100 else c/100))) for c,cs in groupby(coins))))

your change for  9.42$ =  1×5$  +  4×1$  +  1×25¢ +  1×10¢ +  1×5¢  +  2×1¢ 
your change for  8.90$ =  1×5$  +  3×1$  +  3×25¢ +  1×10¢ +  1×5¢ 
your change for  8.61$ =  1×5$  +  3×1$  +  2×25¢ +  1×10¢ +  1×1¢ 
your change for  4.98$ =  4×1$  +  3×25¢ +  2×10¢ +  3×1¢ 
your change for  3.52$ =  3×1$  +  2×25¢ +  2×1¢ 
your change for  3.96$ =  3×1$  +  3×25¢ +  2×10¢ +  1×1¢ 
your change for  9.99$ =  1×5$  +  4×1$  +  3×25¢ +  2×10¢ +  4×1¢ 
your change for  4.96$ =  4×1$  +  3×25¢ +  2×10¢ +  1×1¢ 
your change for  7.23$ =  1×5$  +  2×1$  +  2×10¢ +  3×1¢ 
your change for  6.07$ =  1×5$  +  1×1$  +  1×5¢  +  2×1¢ 


In [162]:
roman = {  1:  'i',   4: 'iv',    5:  'v',   9: 'ix',  10: 'x',
          40: 'ix',  50:  'x',   90: 'xc', 100:  'c', 400: 'cd',
         500:  'd', 900: 'cm', 1000:  'm',}

for _ in range(10):
    year = randint(1900,2200) # randomly pick a year
    pred = predicate(year) # create the predicate
    
    numerals = greedy(reversed(sorted(roman)), pred) # greedy algorithm
    
    # pretty print
    print('the year {} is written {}'.format( year,''.join(roman[x].upper() for x in list(numerals))))

the year 2056 is written MMXVI
the year 1959 is written MCMXIX
the year 1974 is written MCMXXXIV
the year 1940 is written MCMIX
the year 1942 is written MCMIXII
the year 2198 is written MMCXCVIII
the year 1990 is written MCMXC
the year 1911 is written MCMXI
the year 2092 is written MMXCII
the year 2026 is written MMXXVI


# VII. Showcase

The showcase is one of the main attractions of `The Price is Right.`

70+ games.

Drew Carey has lost a lot of weight. Who remembers Bob Barker? 

The `Price is Right` is a global phenomenon: 購物街 (高博)

Showcase is the last game; spin the wheel.

In [166]:
from itertools import repeat, count, chain
from random import randrange
from time import sleep

# helper function
sleep = lambda n, sleep=sleep: lambda: sleep(n)

randoms = (randrange(0,63) for _ in count())
pauses = chain(repeat(sleep(0), 2500), repeat(sleep(0.01), 250), repeat(sleep(0.1), 25), repeat(sleep(1), 5))

for pause, random in zip(pauses, randoms):
    pause()
    print(random)

15
29
13
54
51
46
15
31
51
22
46
23
42
22
19
49
54
42
0
3
27
43
13
47
5
16
37
53
16
54
9
44
31
1
43
31
20
56
14
20
62
13
59
18
28
44
27
57
24
60
38
15
28
30
9
43
35
30
45
7
12
44
37
52
50
29
31
62
32
46
30
44
33
9
37
18
15
25
22
3
7
1
27
3
28
51
15
52
59
45
26
52
0
44
29
45
0
30
46
54
32
57
54
52
29
42
55
58
33
3
8
55
33
10
35
32
51
15
31
35
57
54
58
22
36
59
7
26
49
58
0
54
41
34
3
13
44
42
10
53
27
29
44
49
25
59
48
30
22
38
34
52
16
30
25
36
32
51
33
24
59
53
21
61
21
51
47
42
35
9
54
21
45
11
49
0
16
5
50
43
58
50
35
26
58
3
26
12
33
55
1
41
20
35
62
57
36
22
22
41
60
41
44
51
21
45
19
54
2
3
33
43
19
25
59
35
40
2
3
41
32
44
7
14
52
60
35
44
20
32
43
26
29
8
41
51
35
11
14
58
35
24
44
2
40
29
33
1
45
38
43
53
60
5
62
26
55
62
21
2
44
54
7
51
32
23
51
39
9
21
41
54
53
34
52
11
10
44
50
12
50
9
59
30
21
2
28
42
28
0
52
38
24
47
62
61
19
49
48
61
53
8
52
47
61
4
50
19
61
15
35
54
29
53
23
3
43
5
12
24
52
13
31
31
21
54
5
13
27
42
59
42
40
13
3
57
62
44
0
31
58
50
27
21
46
26
18
26
39

In [168]:
# ref: en.wikipedia.org/wiki/Mersenne_twister
def mersenne(seed = 1, period=397, length=624):
    state, tm = [seed & 0xffffffff], lambda op, x: x ^ op(x)
    for i in range(1,length):
        state.append((0x6c078965 * (state[-1] ^ (state[-1] >> 30)) + i) & 0xffffffff)
    while True:
        for i in range(length):
            y = (state[i] & 0x80000000) + (state[(i+1)%length] & 0x7fffffff)
            state[i] = (state[(i+period)%length] ^ (y >> 1)) ^ (0x9908b0df if y%2 else 0)
        for i in range(length):
            yield tm(lambda x: x >> 18, tm(lambda x: (x << 15) & 0xefc60000, tm(lambda x: (x << 7) & 0x9d2c5680, tm(lambda x: x >> 11, state[i]))))
 
from itertools import takewhile
def randrange(start, stop, mersenne=mersenne(seed=1)):
    size = 2**32 // (stop - start)
    return start + next(takewhile(lambda x: 0 <= x < (stop-start)*size,mersenne)) % (stop-start)

m = mersenne()
assert list(islice(m,0,10)) == [1791095845, 4282876139, 3093770124, 4005303368, 491263, 550290313, 1298508491, 4290846341, 630311759, 1013994432]

In [170]:
from random import randrange

class Wheel(object):
    def __init__(self, start=0, players=100):
        self.start = start
        self.players = players
        
    def spin(self):
        state = self.start
        for _ in xrange(randrange(100,200)):
            print('| {:>2f} |'.format(self.state))
            self.state = (self.state + 1) % players
    
    # ...

In [172]:
from __future__ import division
from time import sleep
from random import random

forward, backward = lambda n: n+1, lambda n: n-1
def wheel(iterable, state=0, direction=forward):
    values = list(iterable)
    while True:
        direction = (yield values[state]) or direction
        state = direction(state) % len(values)

transition = lambda v,t: v-v/abs(v)*5*random()-abs(v)/(2+random())*t
def spin(wheel, velocity=500, stop=0.25, transition=transition):
    next(wheel) # ugly
    while abs(velocity) > stop:
        yield wheel.send(forward if velocity > 0 else backward)     
        sleep(1/abs(velocity))
        velocity = transition(velocity,1/velocity)

In [173]:
def spin(wheel, velocity=500, stop=0.25, transition=transition):
    next(wheel) # ugly
    while abs(velocity) > stop:
        yield wheel.send(lambda n: n + int(velocity)) # <--
        sleep(1/abs(velocity))
        velocity = transition(velocity,1/velocity)

In [176]:
w = wheel([0,1,2])
assert next(w) == 0
assert next(w) == 1
assert next(w) == 2
assert next(w) == 0
assert next(w) == 1

w = wheel([0,1,2], direction=backward)
assert next(w) == 0
assert next(w) == 2
assert next(w) == 1
assert next(w) == 0
assert next(w) == 2

w = wheel([0,1,2], direction=lambda n: n+3)
assert next(w) == 0
assert next(w) == 0

w, v = wheel([0,1,2]), wheel(range(3))
assert next(w) == next(v)
assert next(w) == next(v)

w = wheel([0,1,2])
%timeit list(spin(w, velocity=10, stop=10))
%timeit list(spin(w, velocity=10, stop=0, transition=lambda v,t: v-5*v*t))

1.39 µs ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
301 ms ± 17.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [177]:
friction = lambda v,t: v-v/abs(v)*5*random()-abs(v)/(2+random())*t
def velocity(start=500, stop=0.25, friction=friction):
    state = start
    while abs(state) > stop:
        yield state
        state = friction(state, 1/state)

def spin(wheel, velocity):
    next(wheel) # ugly
    for vel in velocity:
        yield wheel.send(lambda n: n + int(vel))
        sleep(1/abs(vel))

In [179]:
v = velocity(500, 0.25)
assert next(v) > next(v)

v = velocity(500, 0.25)
assert list(v)

v = velocity(500, 0.25)
print(list(v))

[500, 495.7432364939592, 490.3110422112246, 488.93024814643235, 485.8982451123479, 484.9884363338923, 483.73111776239597, 478.63586165051214, 477.68108065760305, 475.57111662772417, 474.5121382318619, 472.5842706758932, 471.2468944632499, 465.9210643549413, 463.779712764513, 460.2750288510237, 456.5080008644591, 455.52935685447693, 452.7362966340236, 451.0205424947194, 448.10737853680723, 444.8376442747017, 442.50272217674944, 441.9514348909234, 440.6480214334978, 438.62404744441244, 436.9928380325352, 435.56527634870264, 434.8268328691357, 433.15270307514544, 429.46196357627247, 424.40325421917, 422.74723567829386, 421.5879492737933, 417.7465096334807, 416.4453198240552, 414.30164070086886, 410.1159422502941, 404.96788445328696, 402.70434760016593, 400.66028790879835, 397.1438904794024, 394.9497451604382, 392.5595926924146, 387.5055864595757, 383.58982341853806, 380.9832331253032, 377.0376011279511, 371.92968217679964, 370.1171300829045, 369.03075008394364, 364.817864202328, 360.38711

All together...

In [180]:
from __future__ import division
from time import sleep
from random import random

forward, backward = lambda n: n+1, lambda n: n-1
def wheel(iterable, state=0, direction=forward):
    values = list(iterable)
    while True:
        direction = (yield values[state]) or direction
        state = direction(state) % len(values)

friction = lambda v,t: v-v/abs(v)*5*random()-abs(v)/(2+random())*t
def velocity(start=500, stop=0.25, friction=friction):
    state = start
    while abs(state) > stop:
        yield state
        state = friction(state, 1/state)

def spin(wheel, velocity):
    next(wheel) # ugly
    for vel in velocity:
        yield wheel.send(lambda n: n + int(vel))
        sleep(1/abs(vel))

In [182]:
from random import randrange, shuffle

players, prizes = range(25), ['book'] + ['t-shirt']*5 + ['mug']*5
winners = [(player, prize) for player in players for prize in prizes]
shuffle(winners)

state, velocity = randrange(0, len(winners)), velocity(randrange(500, 750))

for number, prize in spin(wheel(winners, state=state), velocity=velocity):
    print('| {:>2}  {:<{}} |'.format(number, prize, max(len(p) for p in prizes)))

print('you won a ... {}, #{}'.format(prize, number))


|  4  mug     |
| 16  mug     |
| 11  t-shirt |
| 20  t-shirt |
|  3  mug     |
|  7  mug     |
|  8  t-shirt |
|  6  t-shirt |
|  8  mug     |
|  5  t-shirt |
|  4  mug     |
| 11  t-shirt |
|  5  t-shirt |
| 18  mug     |
|  5  mug     |
| 24  t-shirt |
| 19  book    |
|  0  mug     |
|  3  book    |
|  5  t-shirt |
| 15  t-shirt |
| 24  t-shirt |
|  1  mug     |
|  4  book    |
| 20  t-shirt |
| 21  mug     |
|  9  t-shirt |
|  9  mug     |
| 10  mug     |
|  8  mug     |
| 10  t-shirt |
|  1  mug     |
|  1  mug     |
|  5  mug     |
| 17  mug     |
| 17  t-shirt |
| 16  mug     |
|  4  t-shirt |
| 18  mug     |
| 18  t-shirt |
| 16  book    |
| 19  t-shirt |
| 11  t-shirt |
| 22  mug     |
| 20  t-shirt |
| 19  mug     |
| 17  mug     |
| 16  t-shirt |
| 20  t-shirt |
| 16  t-shirt |
| 21  t-shirt |
| 20  t-shirt |
|  4  mug     |
| 24  book    |
| 15  t-shirt |
|  2  mug     |
|  9  t-shirt |
|  2  t-shirt |
| 22  t-shirt |
| 21  mug     |
|  7  t-shirt |
| 23  mug     |
| 10  mu